In [3]:
import os
home_dir=os.path.expanduser('~')
os.chdir(f"{home_dir}/nexus_correlation_discovery/")
# from nexus.utils.time_point import T_GRANU
# from nexus.utils.coordinate import S_GRANU
from nexus.utils.spatial_hierarchy import SPATIAL_GRANU
from nexus.utils.time_point import TEMPORAL_GRANU
from demo.demo_ui import show_df
from nexus.nexus_api import API
from sklearn import linear_model
import warnings
from nexus.corr_analysis.graph.graph_utils import filter_on_signals
import networkx as nx
from nexus.utils.io_utils import load_corrs_from_dir
from demo.cluster_utils import CorrCommunity
from demo.demo_ui import show_communities
import pickle

warnings. filterwarnings('ignore')

use_qgrid = False

In [ ]:

conn_str = "postgresql://yuegong@localhost/chicago_1m_new"
nexus_api = API(conn_str, data_sources=['chicago_1m', 'chicago_factors'])
dataset = 'chicago_census_tract_population'
t_granu, s_granu = None, S_GRANU.TRACT
overlap_t = 10
r_t = 0.6
df = nexus_api.find_correlations_from(dataset, t_granu, s_granu, overlap_t, r_t, corr_type="pearson")
print(len(df))

In [ ]:
conn_str = "postgresql://yuegong@localhost/chicago_1m_zipcode"
nexus_api = API(conn_str, data_sources=['chicago_1m_zipcode', 'chicago_factors'])
print(nexus_api.get_total_number_of_vars())
# dataset = 'chicago_zipcode_population'
dataset = 'chicago_income_by_zipcode'
t_granu, s_granu = None, S_GRANU.ZIPCODE
overlap_t = 10
r_t = 0
df = nexus_api.find_correlations_from(dataset, t_granu, s_granu, overlap_t, r_t, corr_type="pearson")
print(len(df))

# Test controlling for varaibles

In [ ]:
dataset = 'asthma'
t_granu, s_granu = None, S_GRANU.ZIPCODE
overlap_t = 5
r_t = 0.5
control_vars = [Var('chicago_zipcode_population_zipcode_6', 'avg_population')]
df = nexus_api.find_correlations_from(dataset, t_granu, s_granu, overlap_t, r_t, corr_type="pearson", control_vars=control_vars)
print(len(df))

# Test factor analysis on correlation matrix

In [ ]:
from corr_analysis.graph.graph_utils import filter_on_signals
import networkx as nx
from utils.io_utils import load_corrs_from_dir
from corr_analysis.factor_analysis.factor_analysis import corr_matrix_from_corrs
corrs, corr_map = load_corrs_from_dir('evaluation/correlations2/chicago_1m_T_GRANU.MONTH_S_GRANU.TRACT/')
print(len(corrs))
corrs = filter_on_signals(corrs, None, [1.0, 1.0, 1.0, 0.8, 0.6, 70])
df = corr_matrix_from_corrs(corrs, corr_map)

In [ ]:
import pandas as pd
from factor_analyzer import FactorAnalyzer
import pickle
fa = FactorAnalyzer(n_factors=10, rotation=None, is_corr_matrix=True)
fa.fit(df)
pickle.dump(fa, open('fa.pkl', 'wb'))

In [ ]:
import matplotlib.pyplot as plt
ev, v = fa.get_eigenvalues()
print(len([x for x in ev if x > 1]))
xvals = range(1, df.shape[1]+1)
plt.scatter(xvals, ev)
plt.plot(xvals, ev)

In [ ]:
communalities = fa.get_communalities()
communalities_df = pd.DataFrame(fa.get_communalities(),index=df.columns,columns=['Communalities'])

In [ ]:
loadings_df = pd.DataFrame(fa.loadings_,index=df.columns,columns=['Factor1','Factor2','Factor3','Factor4','Factor5','Factor6','Factor7','Factor8','Factor9','Factor10'])

In [ ]:
clusters = {}
for i in range(10):
    clusters[i] = loadings_df[loadings_df[f'Factor{i+1}'] >= 0.5].index
    print(len(clusters[i]))

In [ ]:
from utils.io_utils import load_corrs_from_dir
df2, _ = load_corrs_from_dir('tmp/chicago_open_data_tract/')

In [2]:
import duckdb

conn = duckdb.connect('data/quickstart.db')
df = conn.execute("SELECT * FROM 'xhc6-88s9_location_6'").df()
print(len(df))


58


In [ ]:
res = conn.sql("SELECT array_length(spatio_temporal_keys) from 'space_6_inv' where val = '60602'").df()
print(res)

In [ ]:
import geopandas as gpd
path = "resource/chicago_shapes/shape_chicago_zipcodes/geo_export_a86acac7-4554-4a8c-b482-7e49844799cf.shp"
gdf = gpd.read_file(path)
print(gdf.head())